## Importing Libraries

In [ ]:
import lightningchart as lc 
import pandas as pd
import numpy as np
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

lc.set_license("LICENSE-KEY")

## Accessing and Downloading Sensor Data

In [ ]:
client = Client("GFZ")
starttime = UTCDateTime("2016-07-12T10:00:00")
endtime = UTCDateTime("2016-07-12T11:00:00")

st = client.get_waveforms(network="1C", station="WAR1", location="--", channel="HHZ", starttime=starttime, endtime=endtime)

# Inspect the metadata to primarily check for the number of the datapoints
tr = st[0]
print(tr.stats)

## Uptime of Each Sensor

In [ ]:
# Defining the uptime hours based on the abstract text provided in the source
uptime_hours = {
    'station': ['WAR1', 'WAR2', 'WAR3', 'WAR4', 'WAR5', 'WAR6', 'WAR7', 'WAR8', 'WAR9'],
    'uptime_hours': [41 * 24, 49 * 24, 49 * 24, 49 * 24, 49 * 24, 49 * 24, 47 * 24, 49 * 24, 49 * 24],
    'total_hours': 49 * 24
}

uptime_df = pd.DataFrame(uptime_hours)
uptime_df['uptime_percentage'] = (uptime_df['uptime_hours'] / uptime_df['total_hours']) * 100

# Preparing the data for the LightningChart bar chart
data = [{'category': station, 'value': uptime} for station, uptime in zip(uptime_df['station'], uptime_df['uptime_percentage'])]


chart = lc.BarChart(
    vertical=True,
    theme=lc.Themes.Dark,
    title='Sensor Uptime Percentage by Station'
)
chart.set_sorting('disabled')
chart.set_data(data)
chart.open()


## Visualization of a Single Sensor’s Raw Waveform Data

In [ ]:
x_values_seconds = st[0].times().tolist()
start_time = st[0].stats.starttime.timestamp * 1000

# If the time is in UTC+3, subtract 3 hours (10800 seconds)
offset_seconds = 3 * 3600
x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds]

y_values = st[0].data.tolist()

chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title="Seismic Data from WAR1"
)

chart.get_default_x_axis().dispose()
chart.get_default_y_axis().set_title("Raw count")

x_axis = chart.add_x_axis(axis_type='linear-highPrecision')
x_axis.set_tick_strategy('DateTime')
x_axis.set_scroll_strategy('progressive')
x_axis.set_title("Time")

series = chart.add_line_series().append_samples(
    x_values=x_values,
    y_values=y_values
)

series.set_line_thickness(2)

chart.open()


### Waveform Data for all Components (HHZ, HHE, HHN)

In [ ]:
client = Client("GFZ")

starttime = starttime
endtime = endtime
station = "WAR1"

# Fetch waveforms for all three components
st = client.get_waveforms(network="1C", station=station, location="--", channel="HH?", starttime=starttime, endtime=endtime)
st.merge(method=1)

# Separate components
tr_z = st.select(channel="HHZ")[0]
tr_e = st.select(channel="HHE")[0]
tr_n = st.select(channel="HHN")[0]

times = tr_z.times().tolist()
data_z = tr_z.data.tolist()
data_e = tr_e.data.tolist()
data_n = tr_n.data.tolist()

chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title='Seismic Waveform for All WAR1 Components'
)

waveform_series_z = chart.add_line_series().append_samples(times, data_z).set_name('HHZ').set_line_thickness(2)
waveform_series_e = chart.add_line_series().append_samples(times, data_e).set_name('HHE').set_line_thickness(2)
waveform_series_n = chart.add_line_series().append_samples(times, data_n).set_name('HHN').set_line_thickness(2)

chart.open()


## Multiple Sensors’ Data Visualization (Stacked Chart)

In [ ]:
client = Client("GFZ")
starttime = starttime
endtime = endtime

# List of stations
stations = [f"WAR{i}" for i in range(1, 10)]

# Dictionary to hold data
data = {}


for station in stations:
        st = client.get_waveforms(network="1C", station=station, location="--", channel="HHZ", starttime=starttime, endtime=endtime)
        data[station] = st[0]

chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title="Seismic Data Comparison (WAR1 to WAR9)"
)

chart.get_default_x_axis().dispose()
chart.get_default_y_axis().dispose()

x_axis = chart.add_x_axis(axis_type='linear-highPrecision')
x_axis.set_tick_strategy('DateTime')
x_axis.set_scroll_strategy('progressive')
x_axis.set_title("Time")

# Create and add stacked y-axes and series for each station
for i, station in enumerate(stations):
    axis_y = chart.add_y_axis(stack_index=i)
    axis_y.set_margins(15 if i > 0 else 0, 15 if i < len(stations) - 1 else 0)
    axis_y.set_title(title=station)
    #axis_y.set_interval()

    trace = data[station]
    x_values_seconds = trace.times().tolist()
    start_time = trace.stats.starttime.timestamp * 1000
    offset_seconds = 3 * 3600  # Offset for UTC+3
    x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds]
    y_values = trace.data.tolist()

    # Add a line series for each station
    series = chart.add_line_series(y_axis=axis_y, data_pattern='ProgressiveX')
    series.append_samples(x_values, y_values)
    
    series.set_line_thickness(2)

chart.open()


## Velocity Visualization

In [ ]:
client = Client("GFZ")
starttime = starttime
endtime = endtime

stVel = client.get_waveforms(network="1C", station="WAR1", location="", channel="HHZ", starttime=starttime, endtime=endtime)

# Fetch response information
inv = client.get_stations(network="1C", station="WAR1", location="", channel="HHZ", level="response")

stVel.remove_response(inventory=inv, output="VEL")

x_values_seconds = stVel[0].times().tolist()
start_time = stVel[0].stats.starttime.timestamp * 1000

# If the time is in UTC+3, subtract 3 hours (10800 seconds)
offset_seconds = 3 * 3600
x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds]
y_values = stVel[0].data.tolist()

chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title="Velocity Data from WAR1"
)

chart.get_default_x_axis().dispose()
chart.get_default_y_axis().set_title("Velocity (m/s)")

x_axis = chart.add_x_axis(axis_type='linear-highPrecision')
x_axis.set_tick_strategy('DateTime')
x_axis.set_scroll_strategy('progressive')
x_axis.set_title("Time")

series = chart.add_line_series().append_samples(
    x_values=x_values,
    y_values=y_values
)

series.set_line_thickness(2)

chart.open()


## Displacement Visualization

In [ ]:
client = Client("GFZ")
starttime = starttime
endtime = endtime

stDis = client.get_waveforms(network="1C", station="WAR1", location="", channel="HHZ", starttime=starttime, endtime=endtime)

# Fetch response information
inv = client.get_stations(network="1C", station="WAR1", location="", channel="HHZ", level="response")

stDis.remove_response(inventory=inv, output="DISP")

x_values_seconds = stDis[0].times().tolist()
start_time = stDis[0].stats.starttime.timestamp * 1000

offset_seconds = 3 * 3600
x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds]
y_values = stDis[0].data.tolist()

chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title="Displacement Data from WAR1"
)

chart.get_default_x_axis().dispose()
chart.get_default_y_axis().set_title("Displacement (m)")

x_axis = chart.add_x_axis(axis_type='linear-highPrecision')
x_axis.set_tick_strategy('DateTime')
x_axis.set_scroll_strategy('progressive')
x_axis.set_title("Time")

series = chart.add_line_series().append_samples(
    x_values=x_values,
    y_values=y_values
)

series.set_line_thickness(2)

chart.open()


## Acceleration Visualization

In [ ]:
client = Client("GFZ")
starttime = starttime
endtime = endtime

stAcc = client.get_waveforms(network="1C", station="WAR1", location="", channel="HHZ", starttime=starttime, endtime=endtime)

# Fetch response information
inv = client.get_stations(network="1C", station="WAR1", location="", channel="HHZ", level="response")

stAcc.remove_response(inventory=inv, output="ACC")

x_values_seconds = stAcc[0].times().tolist()
start_time = stAcc[0].stats.starttime.timestamp * 1000

offset_seconds = 3 * 3600
x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds]
y_values = stAcc[0].data.tolist()

chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title="Acceleration Data from WAR1"
)

chart.get_default_x_axis().dispose()
chart.get_default_y_axis().set_title("Acceleration (m/s²)")

x_axis = chart.add_x_axis(axis_type='linear-highPrecision')
x_axis.set_tick_strategy('DateTime')
x_axis.set_scroll_strategy('progressive')
x_axis.set_title("Time")

series = chart.add_line_series().append_samples(
    x_values=x_values,
    y_values=y_values
)

series.set_line_thickness(2)

chart.open()

## Waveform Data + Velocity, Displacement and Acceleration Visualization (Dashboard)

In [ ]:
dashboard = lc.Dashboard(columns=2, rows=2, theme=lc.Themes.Dark)

# Common time offset
offset_seconds = 3 * 3600

# Helper function to calculate x_values
def calculate_x_values(stream, offset_seconds):
    x_values_seconds = stream[0].times().tolist()
    start_time = stream[0].stats.starttime.timestamp * 1000
    x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds]
    return x_values

# Calculate x_values for all data series
x_values0 = calculate_x_values(st, offset_seconds)
x_values1 = calculate_x_values(stVel, offset_seconds)
x_values2 = calculate_x_values(stDis, offset_seconds)
x_values3 = calculate_x_values(stAcc, offset_seconds)

# Prepare the y_values for all data series
y_values0 = st[0].data.tolist()
y_values1 = stVel[0].data.tolist()
y_values2 = stDis[0].data.tolist()
y_values3 = stAcc[0].data.tolist()

dashboard = lc.Dashboard(columns=2, rows=2, theme=lc.Themes.Dark)

# Seismic waveform chart
chart0 = dashboard.ChartXY(column_index=0, row_index=0, column_span=1, row_span=1, title="Seismic Waveform Data from WAR1")
chart0.get_default_x_axis().dispose()
chart0.get_default_y_axis().set_title("Raw count")

x_axis0 = chart0.add_x_axis(axis_type='linear-highPrecision')
x_axis0.set_tick_strategy('DateTime')
x_axis0.set_scroll_strategy('progressive')
x_axis0.set_title("Time")

series0 = chart0.add_line_series().append_samples(
    x_values=x_values0,
    y_values=y_values0
)
series0.set_line_thickness(2)

# Velocity chart
chart1 = dashboard.ChartXY(column_index=1, row_index=0, column_span=1, row_span=1, title="Velocity Data from WAR1")
chart1.get_default_x_axis().dispose()
chart1.get_default_y_axis().set_title("Velocity (m/s)")

x_axis1 = chart1.add_x_axis(axis_type='linear-highPrecision')
x_axis1.set_tick_strategy('DateTime')
x_axis1.set_scroll_strategy('progressive')
x_axis1.set_title("Time")

series1 = chart1.add_line_series().append_samples(
    x_values=x_values1,
    y_values=y_values1
)
series1.set_line_thickness(2)

# Displacement chart
chart2 = dashboard.ChartXY(column_index=0, row_index=1, column_span=1, row_span=1, title="Displacement Data from WAR1")
chart2.get_default_x_axis().dispose()
chart2.get_default_y_axis().set_title("Displacement (m)")

x_axis2 = chart2.add_x_axis(axis_type='linear-highPrecision')
x_axis2.set_tick_strategy('DateTime')
x_axis2.set_scroll_strategy('progressive')
x_axis2.set_title("Time")

series2 = chart2.add_line_series().append_samples(
    x_values=x_values2,
    y_values=y_values2
)
series2.set_line_thickness(2)

# Acceleration chart
chart3 = dashboard.ChartXY(column_index=1, row_index=1, column_span=1, row_span=1, title="Acceleration Data from WAR1")
chart3.get_default_x_axis().dispose()
chart3.get_default_y_axis().set_title("Acceleration (m/s²)")

x_axis3 = chart3.add_x_axis(axis_type='linear-highPrecision')
x_axis3.set_tick_strategy('DateTime')
x_axis3.set_scroll_strategy('progressive')
x_axis3.set_title("Time")

series3 = chart3.add_line_series().append_samples(
    x_values=x_values3,
    y_values=y_values3
)
series3.set_line_thickness(2)

dashboard.open()

## Additional Visualizations

### Area Chart

In [ ]:
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title='Seismic Activity Over Time for Three Sensors'
)

client = Client("GFZ")
starttime = starttime
endtime = endtime
stations = ["WAR1", "WAR2", "WAR4"]
colors = [(255, 215, 0), (0, 255, 255), (255, 0, 255)]  # Yellow for WAR1, cyan for WAR2, magenta for WAR4


for i, station in enumerate(stations):
    st = client.get_waveforms(network="1C", station=station, location="--", channel="HHZ", starttime=starttime, endtime=endtime)
    tr = st[0]
    times = tr.times()
    data = tr.data

    series = chart.add_area_series()
    series.add(times, data)
    series.set_name(f'{station} Activity')
    series.set_line_color(lc.Color(*colors[i]))

chart.open()


### Bipolar Chart

In [ ]:
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title='Comparison of Seismic Activity'
)

client = Client("GFZ")
starttime1 = UTCDateTime("2016-07-12T09:30:00")
endtime1 = UTCDateTime("2016-07-12T10:30:00")
starttime2 = UTCDateTime("2016-07-12T10:30:00")
endtime2 = UTCDateTime("2016-07-12T11:30:00")
station = "WAR1"

st1 = client.get_waveforms(network="1C", station=station, location="--", channel="HHZ", starttime=starttime1, endtime=endtime1)
st2 = client.get_waveforms(network="1C", station=station, location="--", channel="HHZ", starttime=starttime2, endtime=endtime2)

tr1 = st1[0]
tr2 = st2[0]

times1 = tr1.times()
data1 = tr1.data

times2 = tr2.times()
data2 = tr2.data

# Ensure both datasets have the same length by trimming the longer one
min_length = min(len(data1), len(data2))
data1 = data1[:min_length]
data2 = data2[:min_length]
times = times1[:min_length]

series = chart.add_bipolar_area_series().add(times, data1 - data2)
series.set_name('Seismic Activity Difference')

chart.open()
